# Modelos para probabilidad de respuesta

Importamos librerías:

In [23]:
import os
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

In [3]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Cambiamos directorio de trabajo:

In [28]:
os.chdir('C:/Users/celin/OneDrive/Documentos/CELINE/FACULTAD/Maestría en Estadística Aplicada/4to semestre/Tesis/Nonresponse-bias-eph')
print(os.getcwd())

C:\Users\celin\OneDrive\Documentos\CELINE\FACULTAD\Maestría en Estadística Aplicada\4to semestre\Tesis\Nonresponse-bias-eph


Cargamos la base de datos individual y hogar:

In [29]:
individual_NEA = pd.read_table("Bases/individual_NEA.txt", delimiter = ",", low_memory=False)
individual_NEA.head()

,ANO4,TRIMESTRE,AGLOMERADO,CODUSU,NRO_HOGAR,REGION_DESC,REGION_COD,CH04,CH06,COMPONENTE,...,ADECCFR,PONDIH,V178,adequi,AGLO_DESC,CBA,CBT,ice,hogar_pobre,hogar_indigente
0,2016,2,7,TQRMNOPPQHMLKSCDEFNFF00471826,1,Noreste,41,1,45,1,...,10,313,NaN,1.00,Posadas,1407.8400,3214.916667,2.283581,0.0,0.0
1,2016,2,7,TQRMNOPPQHMLKSCDEFNFF00471826,1,Noreste,41,2,21,3,...,10,313,NaN,0.76,Posadas,1069.9584,2443.336667,2.283581,0.0,0.0
2,2016,2,7,TQRMNOPPQHMLKSCDEFNFF00471826,1,Noreste,41,2,59,2,...,10,313,NaN,0.76,Posadas,1069.9584,2443.336667,2.283581,0.0,0.0
3,2016,2,7,TQRMNOPPRHMLKSCDEFNFF00471827,1,Noreste,41,2,77,2,...,06,267,NaN,0.63,Posadas,886.9392,2025.397500,2.283581,0.0,0.0
4,2016,2,7,TQRMNOPPSHMLKSCDEFNFF00471828,1,Noreste,41,1,31,2,...,,0,NaN,1.00,Posadas,1407.8400,3214.916667,2.283581,1.0,1.0
